<a href="https://colab.research.google.com/github/Hyorim-Kim/numpi/blob/main/tfc26rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 문자열 토큰 처리 후 LSTM으로 감성분류(?)
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense

In [10]:
samples = ['The cat say on the mat.', 'The dog ate my homework']

token_index = {}
for sam in samples:
  for word in sam.split(sep=' '):
    if word not in token_index:
      token_index[word] = len(token_index)
print(token_index)
print()
tokenizer = Tokenizer(num_words=10)
tokenizer.fit_on_texts(samples)
token_seq = tokenizer.texts_to_sequences(samples)  # 텍스트를 정수 인덱싱해서 리스트로 반환
print(token_seq)
print(tokenizer.word_index)
print()
token_mat = tokenizer.texts_to_matrix(samples, mode='binary')  # 'count', 'tfidf', 'freq'
print(token_mat)  # 원핫 이진벡터 형태
print(tokenizer.word_counts)
print(tokenizer.document_count)
print(tokenizer.word_docs)

from keras.utils import to_categorical
token_seq = to_categorical(token_seq[0], num_classes=6)
print(token_seq)


{'The': 0, 'cat': 1, 'say': 2, 'on': 3, 'the': 4, 'mat.': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
{'the': 1, 'cat': 2, 'say': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}

[[0. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 1. 1. 1.]]
OrderedDict([('the', 3), ('cat', 1), ('say', 1), ('on', 1), ('mat', 1), ('dog', 1), ('ate', 1), ('my', 1), ('homework', 1)])
2
defaultdict(<class 'int'>, {'on': 1, 'the': 2, 'cat': 1, 'mat': 1, 'say': 1, 'dog': 1, 'homework': 1, 'ate': 1, 'my': 1})
[[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [17]:
docs = ['너무 재밌네요', '최고예요', '참 잘 만든 영화예요', '추천하고 싶은 영화입니다', '한 번 더 보고싶어요',
              '글쎄요', '별로예요', '생각보다 지루하네요', '연기가 어색해요', '재미없어요']
labels = np.array([1,1,1,1,1,0,0,0,0,0])

token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

x = token.texts_to_sequences(docs)  # 토큰화
print('정수 인덱싱된 토큰 결과:', x)

from keras.utils import pad_sequences  # 서로 다른 크기의 데이터를 일정한 크기로 만듦
from keras.layers import Embedding, Flatten
padded_x = pad_sequences(x, 5)
print(padded_x)

word_size = len(token.word_index) + 1  # 임베딩에 입력될 단어수(토큰수)를 지정 : 가능한 토큰 수는 최대값 + 1을 준다.
model = Sequential()
# Embedding(가능토큰수, 벡터크기, input_length=시퀀스개수) - 워드 임베딩이란 텍스트 내의 단어들을 밀집벡터(dense vector)로 만드는 것
# 워드 임베딩 작업을 수행하고 (number of samples, embedding word dimensionality, input_length)인 3D 텐서를 리턴한다.
model.add(Embedding(word_size, 8, input_length=5))  # RNN layer
model.add(LSTM(units=32, activation='tanh'))
# model.add(Flatten())  # embedding 사용했으니 flatten 빼고해도 됨
model.add(Dense(units=32, activation='relu'))  # Dense layer(완전연결층)
model.add(Dense(units=1, activation='sigmoid'))
print(model.summary())

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, labels, epochs=20, verbose=1)
print('eval : %.4f'%(model.evaluate(padded_x, labels)[1]))

print('predict : ', np.where(model.predict(padded_x) > 0.5, 1, 0).ravel())
print('real : ', labels)


{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한': 11, '번': 12, '더': 13, '보고싶어요': 14, '글쎄요': 15, '별로예요': 16, '생각보다': 17, '지루하네요': 18, '연기가': 19, '어색해요': 20, '재미없어요': 21}
정수 인덱싱된 토큰 결과: [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13, 14], [15], [16], [17, 18], [19, 20], [21]]
[[ 0  0  0  1  2]
 [ 0  0  0  0  3]
 [ 0  4  5  6  7]
 [ 0  0  8  9 10]
 [ 0 11 12 13 14]
 [ 0  0  0  0 15]
 [ 0  0  0  0 16]
 [ 0  0  0 17 18]
 [ 0  0  0 19 20]
 [ 0  0  0  0 21]]
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 5, 8)              176       
                                                                 
 lstm_4 (LSTM)               (None, 32)                5248      
                                                                 
 dense_8 (Dense)             (None, 32)                1056      